In [1]:
import sys
import csv
import os

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import DictVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

## Data Prepreparation

In [7]:
def sentence_to_object(file):

    text = file.readlines();
    array = list()
    dict = {}
    ctr = 0

    for line in text:
        data = line.split("\t")
        if data[0] != "?":
            word = data[0];
            pos = data[len(data)-1]
            pos = get_tags(pos, 1)
            dict[word] = pos.strip()
        elif data[0] == '?':
            ctr += 1
            array.append(dict)
            dict = {}
    
    return array

In [8]:
def get_tags(tag, level):

    if level == 1:
        tag = tag.split("-")[0]
    elif level == 2:
        temp = tag.split("-")[1]
        tag = temp[:2]
    elif level == 3:
        temp = tag.split("-")[1]
        tag = temp[2:]

    return tag

In [9]:
def transform(data):

    dict = {'NN':0, 'PR':0, 'DT':0, 'LM':0, 'CC':0, 'VB':0, 'JJ':0, 'RB':0, 'CD':0, 'TS':0}
    temp = []
    ctr = 0

    for sentence in data:
        for key in sentence:
            ctr += 1
            dict[sentence[key]] += 1

        temp.append(dict)
        dict = {'NN':0, 'PR':0, 'DT':0, 'LM':0, 'CC':0, 'VB':0, 'JJ':0, 'RB':0, 'CD':0, 'TS':0}
     

    return temp

In [10]:
def category_vector():
    input_file = open(os.path.abspath('files/labelled_data.csv'))
    csv_f = csv.reader(input_file)

    cat = []
    column = 0
    ctr = 0

    category = {'entity':1, 'abbreviation':2, 'description':3, 'human':4, 'location':5, 'numeric':6}

    for row in csv_f:
        ctr += 1
        if "Questions" not in row:
            temp = []
            for column in range(1, 2):
                if(row[column] != "Category" and (row[column]) > 0):
                    temp.append(category[row[column].lower()])
            cat.append(temp)
            
#     print cat
    
    input_file.close()
            
    return cat

In [23]:
def get_wh_question(file):
    reader = csv.reader(file)
    
    wh_word = {'alin':1, 'saan':2, 'ano':3, 'kailan':4, 'paano':5, 'sino':6, 'bakit':7 }
    
    vec = []
    
    for row in reader:
#         print row
        sentence = row[0].split(" ")
        temp = []

        for word in sentence:
            if word.lower() in wh_word:
                temp.append(wh_word[word.lower()])
                vec.append(temp)
                break
            elif "-" in word.lower() and word.lower().split("-")[0] in wh_word:
                w = word.lower().split("-")
                print w
                temp.append(wh_word[w[0]])
                vec.append(temp)
                break
            elif word.lower() == "nasaan" or word.lower() == "papaano":
                print word.lower()
                temp.append(wh_word[word.lower()[2:]])
                vec.append(temp)
            else:
                print "No wh-question word!", sentence
        
#     print vec
    
    return vec
        

In [12]:
def pos_vec(file):
    input_file = file
    data = []
    
    data = sentence_to_object(input_file)
    data = transform(data)
    
    input_file.close()
    
    return data

In [9]:
pos_data = pos_vec(open(os.path.abspath('files/dataset_pos.out')))
wh_vector = get_wh_question(open(os.path.abspath('files/labelled_data.csv')))
category = category_vector()

print "Data Length: ", len(pos_data)
print "Wh_Question Length: ", len(wh_vector)
print "Category Length: ", len(category)

NameError: name 'pos_vec' is not defined

In [15]:
X_train, X_test, y_train, y_test = train_test_split(pos_data, category, test_size=0.20, random_state = 42)

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

POS tags

In [55]:
vector = DictVectorizer(sparse=False)
X = vector.fit_transform(X_train)
print X_train
training_set = X

print training_set.shape

[ {'PR': 0, 'VB': 0, 'JJ': 0, 'NN': 0, 'RB': 4, 'CC': 0, 'DT': 0, 'LM': 0, 'TS': 0, 'CD': 0}
 {'PR': 0, 'VB': 0, 'JJ': 0, 'NN': 0, 'RB': 5, 'CC': 0, 'DT': 0, 'LM': 0, 'TS': 0, 'CD': 0}
 {'PR': 2, 'VB': 0, 'JJ': 0, 'NN': 0, 'RB': 2, 'CC': 0, 'DT': 0, 'LM': 0, 'TS': 0, 'CD': 0}
 ...,
 {'PR': 0, 'VB': 0, 'JJ': 0, 'NN': 0, 'RB': 8, 'CC': 0, 'DT': 0, 'LM': 0, 'TS': 0, 'CD': 0}
 {'PR': 1, 'VB': 0, 'JJ': 0, 'NN': 1, 'RB': 1, 'CC': 0, 'DT': 1, 'LM': 0, 'TS': 0, 'CD': 0}
 {'PR': 0, 'VB': 0, 'JJ': 0, 'NN': 0, 'RB': 6, 'CC': 0, 'DT': 0, 'LM': 0, 'TS': 0, 'CD': 0}]
(2464, 10)


Wh-question word

In [53]:
training_set = np.asarray(wh_vector)


print training_set.shape

(2993, 1)


## Training

Support Vector Machine

In [49]:
classifier = OneVsRestClassifier(LinearSVC(C=1.0, random_state=42))
classifier.fit(training_set, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [2554, 2464]

## Testing

In [24]:
testing_data = vector.transform(X_test)
prediction = classifier.predict(testing_data)
accuracy  = classifier.score(testing_data, y_test)

print accuracy

0.417207792208
